In [1]:
import networkx as nx
from directed_graph.graph import Graph, load_graph

/Users/dmitry/Desktop/Projects/concept-tree/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
graph = load_graph("clustered_graph.pickle")

In [3]:
print(repr(graph))

Graph(vertices=22893, edges=87016)


In [4]:
def _convert_to_networkx(graph) -> nx.Graph:
    """
    Converts a custom Graph object to a NetworkX Graph.
    Vertices with the same vertex_type are merged into one logical vertex.
    Edges with the same edge_type are merged into one logical edge.
    """
    nx_graph = nx.Graph()

    for edge in graph.get_edges():
        nx_graph.add_node(edge.agent_1)
        nx_graph.add_node(edge.agent_2)
        nx_graph.add_edge(edge.agent_1, edge.agent_2)

    return nx_graph


In [5]:
nx_graph = _convert_to_networkx(graph)

In [22]:
def get_subgraph_from_vertices(graph: Graph, clique: list[str]) -> Graph:
        subgraph = Graph()
        added_edges = set()
        added_pairs = set()

        clique = list(clique)
        for vertex1 in clique:
            if vertex1 not in subgraph.vertices:
                subgraph.add_vertex(vertex1)

            for vertex2_index in graph.vertex_edges[vertex1]:
                edge = graph.edges[vertex2_index]
                new_edge = (
                     edge.agent_1,
                     edge.agent_2,
                     edge.label
                )
                new_pair = (
                     edge.agent_1,
                     edge.agent_2,
                )
                vertex2 = edge.agent_1 if vertex1 == edge.agent_2 else edge.agent_2
                if vertex1 == vertex2:
                     continue
                if vertex2 in clique:
                    if vertex2 not in subgraph.vertices:
                        subgraph.add_vertex(vertex2)
                    if new_pair not in added_pairs:
                        added_pairs.add(new_pair)
                        subgraph.add_edge(
                            *new_edge
                        )
        return subgraph

In [23]:

cliques = nx.find_cliques(nx_graph)

In [24]:
subgraph = None

for clique in cliques:
    if len(clique) > 6:
        subgraph = get_subgraph_from_vertices(graph, clique)
        break

In [25]:
from directed_graph.visualize_graph import visualize_graph_ngrams_with_pca

visualize_graph_ngrams_with_pca(subgraph)

Excluded vertices due to inconsistent embedding dimensions: set()


In [26]:
print(*subgraph.edges, sep="\n")

Edge(right <--[in]--> state transition)
Edge(thousand <--[of]--> state transition)
Edge(workstation <--[of]--> state transition)
Edge(variance <--[of]--> state transition)
Edge(weighting process <--[demonstrate in]--> state transition)
Edge(synthesis <--[demonstrate of]--> state transition)
Edge(right <--[lie from]--> synthesis)
Edge(right <--[of]--> workstation)
Edge(workstation <--[grant]--> right)
Edge(weighting process <--[group]--> right)
Edge(variance <--[of]--> right)
Edge(thousand <--[of]--> right)
Edge(workstation <--[differentiate]--> variance)
Edge(weighting process <--[demonstrate in]--> workstation)
Edge(workstation <--[in]--> weighting process)
Edge(workstation <--[demonstrate of]--> thousand)
Edge(workstation <--[demonstrate on]--> synthesis)
Edge(thousand <--[of]--> workstation)
Edge(synthesis <--[demonstrate in]--> workstation)
Edge(variance <--[on]--> workstation)
Edge(synthesis <--[of]--> weighting process)
Edge(weighting process <--[add]--> synthesis)
Edge(variance 

In [27]:
from directed_graph.graph import save_graph

save_graph(subgraph, "subgraph.pickle")